In [2]:
import deepsmiles
import mbuild as mb
import foyer
from utils.smiles_utils import convert_smiles, viz, read_comp
import json
import os

In [57]:
def sweep_molecule(smiles_string, test_branch = "CCCC))))",
                   ff_file = "gaff.xml", verbose=False):
    '''
    This function attempts to find which characters in a SMILES string
    would make for good branching sites
    Under the hood, it is using the DeepSMILES format to append
    a test branch structure to each atomic site.
    
    The modified DeepSMILES structure is put under 2 tests:
    1. Try to convert to a standard SMILES string; if so, create an mbuild compound with it.
    2. Try to apply a force field to the mBuild compound
    
    If both are successful, the SMILES string is considered valid and a DeepSMILES formatted
    template is created where after each character that represents a possible bonding site is appended with a
    "{}" 
    
    An example template looks like:
    c{}c{}cc{}cscccsc{}c{}c5c{}s8))))))))c{}c5c{}c9s%12
    
    smiles_string : str
        A valid SMILES representation of a structure
    test_branch : str, default = "CCCC))))"
        A valid DeepSMILES representation of a simple, branch-like structure
    ff_file : str, default = gaff.xml
        A path to a forcefield file that can be used by foyer.Forcefield.apply()
    verbose : bool, default=False
        If true, prints out each SMILES string generated and the status of the attempts.
    '''
    if verbose:
        def verboseprint(*args):
            for arg in args:
                print(arg)
            print()
    else:   
        verboseprint = lambda *a: None 
    
    try:
        deep_smiles_string = convert_smiles(smiles=smiles_string)
    except:
        try:
            smiles_string = convert_smiles(deep=smiles_string)
            deep_smiles_string = smiles_string
        except:
            raise ValueError("{} is not a valid SMILES or DeepSMILES string".format(smiles_string))
    deep_smiles_string_list = list(deep_smiles_string)
    template_list = list(deep_smiles_string)
    forcefield = foyer.Forcefield(os.path.join('force-fields', ff_file))
    
    success_strings = []
    failed_strings = []
    attempts = 0
    success = 0
    for idx, char in enumerate(deep_smiles_string_list):
        if not char.isalpha():
            continue
        attempts += 1
        try_deep_smiles_string = list(deep_smiles_string)
        try_deep_smiles_string.insert(idx+1, test_branch)
        try: # convert to SMILES string
            try_smiles_string = convert_smiles(deep = ''.join(try_deep_smiles_string))
            comp = mb.load(try_smiles_string, smiles=True) 
            try: # apply FF to mbuild compound
                typed_comp = forcefield.apply(comp, assert_dihedral_params=False)
                success_strings.append(try_smiles_string)
                success += 1
                template_list.insert(idx+success, "{}")
                verboseprint("SUCCESS: {}".format(try_smiles_string))
            except:
                verboseprint("FAILED: {}".format(try_smiles_string),
                             "Unable to apply {} force field".format(ff_file))
                failed_strings.append(try_smiles_string)
                continue
        except:
            failed_strings.append(try_deep_smiles_string)
            verboseprint("FAILED: {}".format(try_deep_smiles_string),
                         "Unable to convert from DeepSMILES to SMILES")
            continue
    verboseprint("-------------------------------",
                 "Number of attempted bonds = {}".format(attempts),
                  "Number of successful bonds = {}".format(success),
                "-------------------------------")
    if len(success_strings) == 0:
        return('Not able to create a bonding template')
    return ''.join(template_list)


def create_component_dict(string, name, structure_type,
                          find_bonds=True, full_name = 'Not Provided'):
    '''
    Create a dictionary for a single compound/component using a given SMILES string representation.
    Compound can be created starting with either standard SMILES or DeepSMILES formatting 
    Ultimately, the dictionary will contain both types of SMILES strings. 
    
    string : str, required
        A SMILES string in either standard SMILES or DeepSMILES
    name : str, required
        Generic, identifiable name (Ex. "ITIC", "PTB7", "Benzene")
    structure_type : str, required
        Options are backbone, branch, group. 
    full_name : str, optional, default = "Not Provided"
        Stores the full/official name of the structure 
    ''' 
    d = {}
    try: # string was given in SMILES format
        deep_smiles_string = convert_smiles(smiles = string)
        smiles_string = string
    except: # string was given in DeepSMILES format
        smiles_string = convert_smiles(deep = string)
        deep_smiles_string = string

    if find_bonds:
        template = sweep_molecule(deep_smiles_string, verbose=True)
        print(template)
        d['template'] = template
        
    d['name'] = name
    d['class'] = structure_type
    d['smiles'] = smiles_string
    d['deep_smiles'] = deep_smiles_string
    d['full name'] = full_name
    
    file_name = 'typed-components/{}/{}.json'.format(structure_type, name)
    with open(file_name, 'w') as fp:
        json.dump(d, fp)
    return d



def type_bond_sites(json_file):
    pass
    

def _count_brackets(deep_smiles_string):
    atom_count = 0
    bracket_count = 0
    for s in deep_smiles_string:
        if s.isalpha():
            atom_count += 1    
        if s == ')':
            bracket_count += 1 
    if bracket_count == 0:
        brackets = ')' * atom_count
    elif bracket_count != 0:
        brackets = ')' * (atom_count - bracket_count)
    return brackets

In [58]:
def build_a_branch(branch, groups):
    pass
    


def poly_smiles(monomer_string, length=2):
    '''
    '''
    brackets = _count_brackets(monomer_string)
    monomer_list = list(monomer_string)
    if '*' not in monomer_list: # Check that the polymerization site was specified correctly
        raise ValueError("Identify the wanted polymerization site using *x*'")
    key_indices = [index for index, value in enumerate(monomer_list) if value == '*']  
    if len(key_indices) != 2:   # Checks for only a single given poly site
        raise ValueError("Select only one polymerization site using *x*")
    if key_indices[1] - key_indices[0] != 2:   # Check that the * are surrounding only a single atom
        raise ValueError("Select only one polymerization site using *x*")
    
    # Set up the template string with {} and the correct # of brackets
    monomer_list[key_indices[1]] = '{}' + '{}'.format(brackets) # Create poly site+brackets
    monomer_list.remove('*')
    template = ''.join(monomer_list) # Deepsmiles string with {} at bonding site
    monomer_list.remove('{}' + '{}'.format(brackets))
    monomer = ''.join(monomer_list)  # Deepsmiles monomer string without {} or *
    
    # Loop & format polymer
    polymer = '{}'
    for i in range(0, length):
        if i == length - 1:
            polymer = polymer.format(monomer)
            break
        polymer = polymer.format(template)

    polymer_smiles = convert_smiles(deep = polymer)
    return polymer_smiles

In [66]:
new = 'C1(C=CC(C2=CC=C(C3=CC=CC=C3)C=C2)=C4)=C4OC(C=C(C5=CC=C(C6=CC=CC=C6)C=C5)C=C7)=C7N1'
viz(new, deep=False)
new_deep = convert_smiles(smiles=new)
new_deep = 'cccccccocncc6))))))ccc6))))))ccc6'
viz(new_deep, deep=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [62]:
create_component_dict(string=new_deep,
                      name='phenoxazine',
                      structure_type='backbone',
                      find_bonds=True)

FAILED: ['C', 'CCCC))))', '3', '7', '(', 'C', '=', 'C', 'C', '(', 'C', '2', '=', 'C', 'C', '=', 'C', '(', 'C', '1', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '1', ')', 'C', '=', 'C', '2', ')', '=', 'C', ')', '=', 'C', '-', '3', 'O', 'C', '6', '(', 'C', '=', 'C', '(', 'C', '5', '=', 'C', 'C', '=', 'C', '(', 'C', '4', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '4', ')', 'C', '=', 'C', '5', ')', 'C', '=', 'C', ')', '=', 'C', '-', '6', 'N', '7']
Unable to convert from DeepSMILES to SMILES

FAILED: ['C', '3', '7', '(', 'C', 'CCCC))))', '=', 'C', 'C', '(', 'C', '2', '=', 'C', 'C', '=', 'C', '(', 'C', '1', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '1', ')', 'C', '=', 'C', '2', ')', '=', 'C', ')', '=', 'C', '-', '3', 'O', 'C', '6', '(', 'C', '=', 'C', '(', 'C', '5', '=', 'C', 'C', '=', 'C', '(', 'C', '4', '=', 'C', 'C', '=', 'C', 'C', '=', 'C', '4', ')', 'C', '=', 'C', '5', ')', 'C', '=', 'C', ')', '=', 'C', '-', '6', 'N', '7']
Unable to convert from DeepSMILES to SMILES

FAILED: ['C', '3', '7', '(',

/home/chris/cme/forks/foyer/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)


{'template': 'Not able to create a bonding template',
 'name': 'phenoxazine',
 'class': 'backbone',
 'smiles': 'C37(C=CC(C2=CC=C(C1=CC=CC=C1)C=C2)=C)=C-3OC6(C=C(C5=CC=C(C4=CC=CC=C4)C=C5)C=C)=C-6N7',
 'deep_smiles': 'CC=CCC=CC=CC=CC=CC=C6))))))C=C6))))))=C))))=C-2OCC=CC=CC=CC=CC=CC=C6))))))C=C6))))))C=C))))=C-2N6',
 'full name': 'Not Provided'}

In [10]:
create_component_dict(string='CCCCCC', name="hexane",
                      structure_type='branch', full_name='n-hexane')

/home/chris/cme/forks/foyer/foyer/forcefield.py:336: UserWarning: Parameters have been assigned to all proper dihedrals.  However, there are more parameterized dihedrals (95) than total system dihedrals (81).  This may be due to having multiple periodic dihedrals for a single system dihedral.
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 40, Parameterized impropers: 0. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.py:336: UserWarning: Parameters have been assigned to all proper dihedrals.  However, there are more parameterized dihedrals (97) than total system dihedrals (81).  This may be due to having multiple periodic dihedrals for a single system dihedral.
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.p

C{}C{}C{}C{}C{}C{}


{'template': 'C{}C{}C{}C{}C{}C{}',
 'name': 'hexane',
 'class': 'branch',
 'smiles': 'CCCCCC',
 'deep_smiles': 'CCCCCC',
 'full name': 'n-hexane'}

In [3]:
test_string = "C6c4cc3c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67"
template = sweep_molecule(test_string, ff_file = "gaff.xml", verbose=True)

/home/chris/cme/forks/foyer/foyer/forcefield.py:449: UserWarning: No force field version number found in force field XML file.
  'No force field version number found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/forcefield.py:461: UserWarning: No force field name found in force field XML file.
  'No force field name found in force field XML file.'
/home/chris/cme/forks/foyer/foyer/validator.py:132: ValidationWarning: You have empty smart definition(s)
  warn("You have empty smart definition(s)", ValidationWarning)
/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 148, Parameterized dihedrals: 104. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system 

SUCCESS: C6(CCCC)c4cc3c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67

FAILED: C6c4(CCCC)cc3c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field



/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 145, Parameterized dihedrals: 93. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)


SUCCESS: C6c4c(CCCC)c3c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67

FAILED: C6c4cc3(CCCC)c2sc1ccsc1c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field



/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all proper dihedrals. Total system dihedrals: 153, Parameterized dihedrals: 99. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers.
  warnings.warn(msg)
/home/chris/cme/forks/foyer/foyer/forcefield.py:267: UserWarning: Parameters have not been assigned to all impropers. Total system impropers: 45, Parameterized impropers: 5. Note that if your system contains torsions of Ryckaert-Bellemans functional form, all of these torsions are processed as propers
  warnings.warn(msg)


SUCCESS: C6c4cc3c2(CCCC)sc1ccsc1c2Cc3cc4c7sc5ccsc5c67

FAILED: C6c4cc3c2s(CCCC)c1ccsc1c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

FAILED: C6c4cc3c2sc1(CCCC)ccsc1c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

SUCCESS: C6c4cc3c2sc1c(CCCC)csc1c2Cc3cc4c7sc5ccsc5c67

SUCCESS: C6c4cc3c2sc1cc(CCCC)sc1c2Cc3cc4c7sc5ccsc5c67

FAILED: C6c4cc3c2sc1ccs(CCCC)c1c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

FAILED: C6c4cc3c2scccsc1(CCCC1)c2Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

FAILED: C6c4cc3csc1cc2sc1c(CCCC2)Cc3cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

SUCCESS: C6c4cc3c2sc1ccsc1c2C(CCCC)c3cc4c7sc5ccsc5c67

FAILED: C6c4ccc2sc1c3csc1c2Cc(CCCC3)cc4c7sc5ccsc5c67
Unable to apply gaff.xml force field

SUCCESS: C6c4cc3c2sc1ccsc1c2Cc3c(CCCC)c4c7sc5ccsc5c67

FAILED: C6ccc3c2s4c1ccsc1c2Cc3cc(CCCC4)c7sc5ccsc5c67
Unable to apply gaff.xml force field

SUCCESS: C6c4cc3c2sc1ccsc1c2Cc3cc4c7(CCCC)sc5ccsc5c67

FAILED: C6c4cc3c2sc1ccsc1c2Cc3cc4

In [4]:
print(template)
# Template I created when doing it manually:
print("C{}cc{}ccscc{}c{}sc5c8C{}c%11c{}c%15cscc{}c{}sc5c%248")

C{}cc{}cc{}scc{}c{}sc5c8C{}c%11c{}c%15c{}scc{}c{}sc5c%248
C{}cc{}ccscc{}c{}sc5c8C{}c%11c{}c%15cscc{}c{}sc5c%248


## ---------------------------------------------------------------
### Discarded or WIP stuff
## ---------------------------------------------------------------

In [ ]:
# Using nodes and edges:
def sweep_molecule(smiles_string, test_branch = "CCCC))))", ff_file = "gaff.xml"):
    '''
    This function attempts to find which characters in a SMILES string would make for good branching sites
    Under the hood, it is using the DeepSMILES format to append a test branch structure to each atomic site.
    
    The modified DeepSMILES structure is put under 3 tests:
    1. Try to convert to a standard SMILES string.
    2. Try to create an mBuild compound from the new SMILES string generated from step 1.
    3. Try to apply a force field to the mBuild compound
    
    If all three are successful, the SMILES string is considered valid and a DeepSMILES formatted
    template is created where after each character that represents a possible bonding site is appended with
    "{}" 
    An example template looks like: c{}c{}cc{}cscccsc{}c{}c5c{}s8))))))))c{}c5c{}c9s%12
    
    smiles_string : str
        A valid SMILES representation of a structure
    
    test_branch : str
        A 
    
    '''
    
    deep_smiles_string = convert_smiles(smiles=smiles_string)
    deep_smiles_string_list = list(deep_smiles_string)
    template_list = list(deep_smiles_string)
    
    branch_compound = mb.load(convert_smiles(deep=test_branch), smiles=True)
    backbone_compound = mb.load(smiles_string, smiles=True)
    branch_nodes = branch_compound.bond_graph.number_of_nodes()
    branch_edges = branch_compound.bond_graph.number_of_edges()
    backbone_nodes = backbone_compound.bond_graph.number_of_nodes()
    backbone_edges = backbone_compound.bond_graph.number_of_edges()
    
    forcefield = foyer.Forcefield(os.path.join('force-fields', ff_file))
    success_strings = []
    success = 0
    for idx, char in enumerate(deep_smiles_string_list):
        if not char.isalpha():
            continue
        try_deep_smiles_string = deep_smiles_string_list[:]
        try_deep_smiles_string.insert(idx+1, test_branch)
        try: # convert to SMILES string
            try_smiles_string = convert_smiles(deep = ''.join(try_deep_smiles_string))
            comp = mb.load(try_smiles_string, smiles=True) 
            if (branch_nodes+backbone_nodes-2) == comp.bond_graph.number_of_nodes():
                if branch_edges+backbone_edges-1 == comp.bond_graph.number_of_edges():
                    try: # apply FF to mbuild compound
                        typed_comp = forcefield.apply(comp, assert_dihedral_params=False)
                        success_strings.append(try_smiles_string)
                        success += 1
                        template_list.insert(idx+success, "{}")
                    except:
                        continue
                else:
                    continue
            else:
                continue
        except:
            continue
    return ''.join(template_list), success_strings

In [ ]:
def find_bond_site(smiles_string, deep = True):
    '''    
    '''
    bonding_dicts = []  # List of dictionaries of each bonding site
    smiles_string_list = list(smiles_string)
    template_list = list(smiles_string)
    for index, char in enumerate(smiles_string_list):
        d = {}
        if char.lower() == 'c':  # The index belongs to a carbon site 'c'
            next_ind = index + 1
            try:
                if smiles_string_list[index + 1].isalpha():  # The following character in the string is an atom
                    index_to_change = index  # Change the original site
                    original_value = char
                    smiles_string_list[index_to_change] = original_value + 'N)'
                else:  # The following character is the SMILES string is a symbol or number
                    try:
                        while not smiles_string_list[next_ind].isalpha(): # Find the next alpha containing index
                            next_ind += 1
                        index_to_change = next_ind - 1
                    except:
                        index_to_change = -1  # The SMILES string ends with with a non-alpha value
            
                    original_value = smiles_string_list[index_to_change]  # Add the branch to the last non-alpha
                    smiles_string_list[index_to_change] = original_value + 'N)'
    
            except:
                index_to_change = index # The SMILES string ends with a 'c'
                original_value = char
                smiles_string_list[index_to_change] = original_value + 'N)'
                
            temp_string = ''.join(smiles_string_list)
            viz(temp_string, deep)
            print('Add as possible bonding site? (y/n) or (exit)')  # QUESTION No. 1
            add = input()
            if add.lower() == 'yes' or add.lower() == 'y':
                template_list[index_to_change] = original_value + '{}'
                print('Classify bonding site as (1) branch, (2) polymerization, or (3) both') # QUESTION No. 2
                site_type = input()
                if site_type == '1':
                    bond_site_type = 'branch'
                elif site_type == '2':
                    bond_site_type = 'poly'
                    num_of_bonds = 1
                elif site_type == '3':
                    bond_site_type = 'both'
                if bond_site_type == 'branch' or bond_site_type == 'both':
                    print('How many bonds can be formed from this atom? (1, 2, 3, 4..etc)') # QUESTION No. 3
                    num_of_bonds = int(input())
                d['index'] = index_to_change
                d['type'] = bond_site_type
                d['num_bonds'] = num_of_bonds
                bonding_dicts.append(d)
            if add.lower() == 'exit':
                break
            smiles_string_list[index_to_change] = original_value
        else:
            pass
    template_molecule_string = ''.join(template_list)
    
    return bonding_dicts, template_molecule_string
       

    

In [ ]:
def build_compound(backbone, branches, polymerize=False):
    
    def get_bonding_sites():
        bond_indices = []
        with open('typed-components/{}.json'.format(backbone)) as jf:
            d = json.load(jf)
            bond_info = d['bonding']
            for dictionary in bond_info:
                if not polymerize:
                    bond_indices.append(dictionary['index'])
                elif polyermize:
                    if dictionary['type'] != 'poly':
                        bond_indices.append(dictionary['index'])
        return bond_indices
        
    backbone_smiles = get_smiles_string(backbone)
    backbone_deep_smiles = get_smiles_string(backbone, smi_type='deep_smiles')
    backbone_template = get_smiles_string(backbone, smi_type = 'template')
    branch_smiles = [get_smiles_strings(branch) for branch in branches]
    branch_deep_smiles = [get_smiles_string(branch, smi_type = 'deep_smiles') for branch in branches]
    bond_indices = get_bonding_sites()
    
    print(backbone_smiles)
    print(backbone_deep_smiles)
    print(backbone_template)
    print(branch_smiles)
    print(branch_deep_smiles)
    print(bond_indices)

    
    return bond_indices, branch_smiles

In [ ]:
def poly_smiles_idx(monomer_string, polyindex, length=2, ftype='mol2', string_only=True,
                    energy_min=False, save=False, visualize=False):
    
    monomer_string_d = convert_smiles(smiles=monomer_string)
    
def find_symmetry(string, deep = True):
    '''
    Given a chemical structure, find and classify it's symmetry.
    Groups, axes of symmetry, planes of symmetry, chirality, etc...
    '''
    if deep:
        smiles_string = convert_smiles(string)
    else:
        smiles_string = string
    compound = mb.load(smiles_string, smiles = True)  # mbuild compound
    

In [ ]:
def convert_smiles(smiles=False, deep=False):   
    '''
    smiles and deep must be str format
    Converts from SMILES to DeepSMILES and vice versa.
    Whichever has a string provided, will convert to the other.
    If strings are proivded for both, then nothing happens
    '''
    converter = deepsmiles.Converter(rings=True, branches=True)
    if smiles and deep:
        print('Only provide a string for one of smiles or deep')
        return()
    if smiles: # Convert from SMILES to DeepSMILES
        deep_string = converter.encode(smiles)
        return deep_string
    if deep: # Convert from DeepSMILES to SMILES
        smiles_string = converter.decode(deep)
        return smiles_string 